In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Load origenal data
dataset: data\\DontGetKicked\\training.csv
aim: data\\DontGetKicked\\test.csv


In [ ]:
df = pd.read_csv('data\\DontGetKicked\\training.csv', parse_dates=['PurchDate'], index_col=0)
#df.loc[:, 'PurchDate'] = pd.to_datetime(df.loc[:, 'PurchDate'], unit='s', origin='unix')
display(df.head())
df.info()

# Training-test split
save data in folder: data\\

In [ ]:
from sklearn.model_selection import train_test_split

features = df.drop(columns=['IsBadBuy'])
target = df['IsBadBuy']

features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.15, 
                                                                            random_state=42)

features_train.to_csv('data\\features_train.csv')
target_train.to_csv('data\\target_train.csv')

features_test.to_csv('data\\features_test.csv')
target_test.to_csv('data\\target_test.csv')

In [ ]:
features_train.head()
print(features_train.shape)
print(target_train.shape)
print(target_train.to_frame().shape)

# Make df_train

In [ ]:
df_train = features_train.copy()
df_train.insert(0, 'IsBadBuy', target_train)
df_train.sort_index().head()
df_train.to_csv('data\\df_train.csv')

In [ ]:
df_train.head()


# EDA

In [ ]:
df.columns

In [ ]:
df.plot(x='VehYear', y='VehicleAge', kind='scatter')

In [ ]:
col_year= ['IsBadBuy', 'PurchDate', 'VehYear', 'VehicleAge']
df_year = df[col_year].copy()
df_year.loc[:, 'PurchYear'] = df_year['PurchDate'].dt.year
df_year.loc[:, 'PurchMonth'] = df_year['PurchDate'].dt.month
df_year.loc[:, 'PurchDay'] = df_year['PurchDate'].dt.day
df_year.loc[:, 'Age2'] = df_year['PurchYear'] - df_year['VehYear']
df_year.head()
df_year.plot(x='VehicleAge', y='Age2', kind='scatter')

### conclusion
- strong correlation between 'VehYear' and 'VehicleAge' is related to fact that all cars were bought within 2-years period. 
- PurchYear - 'VehYear' = 'VehicleAge' (just 7 exeptions, month of the year was not considered)
- 'VehYear' can be an additional indication for car model

In [ ]:

df_year.head(8)

In [ ]:
df_year.info()

In [ ]:
# %load_ext autoreload
# %autoreload 2

%reload_ext autoreload

from avp_pckg.DataFrame import AvPdataFrame 
avp_df = AvPdataFrame(df_year)

avp_df.calc_frequency(col='Age2', target='IsBadBuy').sort_index()

In [ ]:
avp_df.calc_frequency(col='PurchMonth', target='IsBadBuy').sort_values('IsBadBuy_%')

In [ ]:
avp_df.calc_frequency(col='PurchDay', target='IsBadBuy').sort_values('IsBadBuy_%')

In [ ]:
avp_df.columns
avp_df.shape

In [ ]:
%load_ext autoreload
%autoreload 2
#reload(AvPdataFrame )
cols = ['VehYear', 'VehicleAge', 'PurchYear', 'PurchMonth', 'PurchDay']
df_impot = avp_df.cols_importance(cols=cols, target='IsBadBuy')
df_impot.head()

In [ ]:
avp_df['IsBadBuy'].sum() / avp_df.shape[0]

### Conclusion
- avearaged IsBadBuy = 12.3 +/- 2.3 % 
- there is no corelation of the IsBadBuy with day, month or year of purch => estimation of random fuctuations 
- => 5% of IsBadBay is a threshold for disctinguishing categories 

In [ ]:
df.columns

In [ ]:
df = AvPdataFrame(df)

In [ ]:
df.isna().sum()
# df.isnull().sum() ### the same result ???


In [ ]:
df.fillna('empty', inplace=True)
df.isna().sum()

In [ ]:
df.dtypes

In [ ]:
### mixed data types leads to error for sorting command
df.loc[:, 'WheelTypeID'] = df['WheelTypeID'].astype('str') ### how to make check in the function?
type(df['WheelTypeID'][1])

In [ ]:
df.calc_frequency('WheelTypeID', 'IsBadBuy')

In [ ]:
cols = ['Auction', 'VehYear', 'VehicleAge', 'Make',
       'Model', 'Trim', 'SubModel', 'Color', 'Transmission', 'WheelTypeID',
       'WheelType', 'Nationality', 'Size', 'TopThreeAmericanName',
       'PRIMEUNIT', 'AUCGUART', 'BYRNO', 'VNZIP1', 'VNST',
       'IsOnlineSale', ]

df.cols_importance(cols=cols, target='IsBadBuy').sort_values('delta1000', ascending=False)

### Conclusion 
- non informative collumns:  Transmission (4 -> 3), IsOnlineSale(2), Nationality (5), Auction(3), TopThreeAmericanName (5)

In [ ]:
df.calc_frequency('WheelTypeID', 'IsBadBuy')

In [ ]:
df.calc_frequency('WheelType', 'IsBadBuy')

count of WheelTypeID 0.0 + empty = 3169 + 5 = 3174 = WheelType empty 
- WheelTypeID is redundency of WheelType column with only 5 additional assignments - will be droped 

In [ ]:
3174
3169 + 5


In [ ]:
df.columns

In [ ]:
df['VNST'].value_counts()
df_TX = df[df['VNST'] == 'TX']

df_TX = df_TX.loc[:, ['VNST', 'VNZIP1', 'BYRNO']].copy()
df_TX['VNZIP1'].value_counts().sort_index()

df_TX_75236 = df_TX[df_TX['VNZIP1'] == 75236 ]
df_TX_75236['BYRNO'].unique()

In [ ]:
# dcols = ['Transmission', 'IsOnlineSale', 'Nationality', 'Auction', 'TopThreeAmericanName', 'WheelTypeID']
# df.drop(columns=dcols, inplace=True)

In [ ]:
num_cols = ['VehOdo', 'VehBCost']